# 01 - Time Series Models

## 01 - Background

Our goal in this challenge is to apply the basic concepts of time series analysis on one-dimension data (sales depending on the date).

In this challenge, we'll go through the following steps : 
1. load and visualize the data;
2. train our models and make predictions;
3. use an econometric approach to model the serie and be able to forecast it;
4. use machine learning to hack this modelization.

The dataset is loaded from [this url]( https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly_champagne_sales.csv). We provide you the code to read the csv from an url. 

## 02 - Data exploration

Start by loading the libraries you need, read the data file and plot some graphs.

You could need to install requests: `pip install requests`.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import io
import requests
url="https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly_champagne_sales.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

## 03 - Convert datetime to index

Now, we should get the index as a datetime : 
- convert the column "Date" to datetime
- reset the index of the DataFrame
- Set the DataFrame index using the "Month" column.
- Drop the column "Month"

In [2]:
# Your code here


## 04 - Visualize and interpet the data

Well done, thanks to this "reindexing", you should now be able to plot the "Sales" (y-axis) as a function of the time (x-axis).

In [4]:
# TO DO : plot the "Sales" as a function of the corresponding time and try to interpret the results. 


If your code is correct, you should be able to see that this Time Serie (TS) is:
- Not stationnary (mean and variance are not constant).
- Exhibits strong saisonality (variations that occur regularly).
- Seems to have a trend.

Let's see a decomposition of the data between **trend**, **saisonality** and **noise**. In order to do that, you have to make use of the [seasonal_decompose from the statsmodels library]. You can install statsmodels with `pip install statsmodels` (https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html). Read the docs and make sure you understand what this function is doing and how to use it. When you feel more comfortable with it:
1. plot the "Sales" with an "additive" model
2. plot the "Sales" with a "multiplicative" model

In [6]:
# TO DO : plot the Sales:
# 1. with an additive model
# 2. with a multiplicative model


- What can you interpret from this first analysis ?

<details>
<summary>
Answer
</summary>
<p>
we can see first the strong seasonality term, that seems to have a coumpound effect. Indeed, the more champagne is sold, the more the seasonality effect can be seen. Also, we can clearly see the drop in champagne sell during 1970-1971.

</p>
</details>





## 05 - Split the data (train/test)
Let's now use a different approach to forecast the data. First we will use an **econometric approach**, and then a more "Machine Learning one".

Let's define how we will measure the errors of the predictions:
- On which data ?
- How do we measure the error ?


The split of the data has to be done "time-wise". For the purpose of this exercise, we will use data up to 1972 for training and after for the test.

Your task have to : 

- split the data before 1972 for the training and after (or equal to) 1972 for the testing
- check the shape of the training and testing dataset

In addition, keep the indexes of the training set in a variable called `train_indexes` and the indexes of the test set in a variable called `test_indexes`.

In [8]:
# TO DO: split the data : 
# df_train : data before 1972
# df_test : data after (and equal to) 1972


In [9]:
from sklearn.metrics import mean_squared_error
import numpy as np
index = df.index
train_indexes = np.where(index < pd.datetime.strptime("1972", "%Y"))[0]
test_indexes = np.where(index >= pd.datetime.strptime("1972", "%Y"))[0]
df_train = df.iloc[train_indexes]
df_test = df.iloc[test_indexes]

In [10]:
# TO DO: output the corresponding shapes of the train and test datasets:


## 06 - Econometric approach

We will analyse the data thanks to [ARIMA models](https://towardsdatascience.com/time-series-forecasting-arima-models-7f221e9eee06) (Auto Regressive Integrated Moving Average).


We need to :
- find how to stationarize the time serie;
- find the moving average;
- find the seasonality and auto-regressive part.

In this exercise, we will follow these steps:

### Step 1: Check stationarity

If a time series has a trend or seasonality component, it must be made stationary before we can use ARIMA to forecast.

Check the documentation of the Augmented Dick Fuller test and check the stationarity of `df["Sales"]` thanks to [this method](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html).


In [12]:
# Your code here


The p-value should be  less than 0.05 to have a 95% confidence in the stationarity. If the p-value is larger than 0.05, we cannot reject the null hypothesis (null hypothesis = "the process is not stationary").

Knowing that, what do you conclude for the "Sales" column?

### Step 2 - Difference

If the time series is not stationary, it needs to be stationarized through *differencing*. It means that we take the difference between each value and the preceding one (if we take the *first difference*) for instance.

Let's start with the first difference. Create a new column `sales_diff` containing the difference between each value and the preceding one.

Then, check for stationarity of the stationarized column using again `adfuller`.


### Step 4 - Select AR and MA terms

You will now use the ACF and PACF plots to decide whether to include an AR term(s), MA term(s), or both.

In order to fit our ARIMA model we need to find the appropriate differenciation to use. The best way to determine whether or not the series is sufficiently differenced is to plot the differenced series and check to see if there is a constant mean and variance. Take as many differences as it takes. Make sure you check seasonal differencing as well.



In [15]:
# Your code here


Then, you can do the following plots:

- The autocorrelation plot applied to the `df["Sales"]` (`plot_acf` from `statsmodels.graphics.tsaplots`).

- The partial autocorrelation applied to the `df["Sales"]` (`plot_pacf` also from `statsmodels.graphics.tsaplots`).


In [17]:
# Your code here


One goal of these plots is to find good values of the parameters `P` and `Q` needed for the ARIMA models. Try to find these values from the plots.


In [19]:
# Your answer here


### Step 5 - Build the model

Now that you have chosen the values for `P` and `Q`, build the `arima_model` (from the `statsmodels` library).

Then, fit the the model.

Store also the parameters of the model (`.params`). You can also print the parameters optimized by the model.

Finally, you can use `plot_predict()` to plot the data along with the prediction. Have a look [at the documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARMAResults.plot_predict.html) if you need.


In [21]:
# TO DO:
# - fit the appropriate ARIMA model with the values P and Q you determined thanks to the ACF and PACF graphs
# - store the params of your model (`.params`)
# - plot the predictions vs actual values


### Step 6 - Validate model

Compare the predicted values to the actuals in the validation sample.

We created the following function for you : `evaluate_performance_month_prediction_arima`. This function evaluates the performance per month of your ARIMA model. Use it on your test dataset to check the performance of your model. From the output, extract the RMSE to measure the errors of your predictions. The function takes the data, the indexes of the test samples (`test_indexes`) and the parameters from above.


In [23]:
def evaluate_performance_month_prediction_arima(df, test_indexes, params):
    predictions = []
    ground_truth = []
    i = 0
    indexes = df.index
#     print("Going to predict on indexes {}".format(test_indexes))
    for _indx in test_indexes:
        train_data = df.iloc[:_indx]
        current_ground_truth = df.iloc[_indx]
        current_date = indexes[_indx]
#         print("iteration number {}, indx: {}, date: {}".format(i, _indx, current_date))
        arima = arima_model.ARIMA(order=(1,1,4), endog=train_data)
#         print("fitting")
        arima_fit = arima.fit(params)
#         print("fitted, predicting")
        prediction = arima_fit.predict(start=current_date,end=current_date).values[0]
#         print("predicted: {}".format(prediction))
        predictions.append(prediction)
        ground_truth.append(current_ground_truth)
        i += 1
#     print("starting rmse computation")
    rmse = np.sqrt(mean_squared_error(ground_truth, predictions))
#     print("ALL done")
    return (ground_truth, predictions, rmse)

In [24]:
# Your code here
# - make use of the function `evaluate_performance_month_prediction_arima`
# - from the output, print the RMSE to evaluate the performance of this model



## 07 - Mix of Econometry and machine learning (SARIMA model)

We saw that the previous ARIMA fitting was not bad, but we can do better.

Now, try to:

- create a new column name "sales_season" and assign it 12-month seasonality thanks to the `diff()` function
- check the stationary of this new column
- just as before, plot the ACF and PACF in order to determine `P` and `Q`

In [26]:
# Your code here


Given the graphs and the test, we can infer that we have an MA(4) AR(1) with seasonality 12. 

We will use machine learning and a brute force method to find the "best" model that fits our training data.

We will find the minimal value for the AIC on a grid of possible values: `[0, 1 or 2]` for the parameters `ma`, `ar`, `seasoned_ma` and `seasoned_ar`. You can use multiple nested for loops and train a `SARIMAX` for each combination and store the aic (`.aic`) in a dictionary along with the corresponding parameters.


In [30]:
# Your code here


Now, find the best combination of parameters: corresponding to the lower AIC value.

In [ ]:
# Your code here


Then, train again the SARIMAX with the best parameters.

In [ ]:
# Your code here


And check the predictions on training sample:

In [ ]:
# Your code here


SARIMA models offer also diagnostics to see if our residuals are Gaussian. Something good to check. You can use `plot_diagnostics()`.

In [ ]:
# Your code here


Now evaluate the performance of this model on the data using the following function `evaluate_performance_month_prediction_sarima()`.

In [ ]:
def evaluate_performance_month_prediction_sarima(df, test_indexes, params):
    predictions = []
    ground_truth = []
    i = 0
    indexes = df.index
#     print("Going to predict on indexes {}".format(test_indexes))
    for _indx in test_indexes:
        train_data = df.iloc[:_indx]
        current_ground_truth = df.iloc[_indx]
        current_date = indexes[_indx]
#         print("iteration number {}, indx: {}, date: {}".format(i, _indx, current_date))
        sarima_model = SARIMAX(df_train["Sales"], order=(_ar_val, 0, _ma_val), seasonal_order=(_seas_ar_val, 0, _seas_ma_val, 12), enforce_invertibility=False, enforce_stationarity=False)
#         print("fitting")
        sarima_model_fit = sarima_model.fit(params)
#         print("fitted, predicting")
        prediction = sarima_model_fit.predict(start=current_date,end=current_date).values[0]
#         print("predicted: {}".format(prediction))
        predictions.append(prediction)
        ground_truth.append(current_ground_truth)
        i += 1
#     print("starting rmse computation")
    rmse = np.sqrt(mean_squared_error(ground_truth, predictions))
#     print("ALL done")
    return (ground_truth, predictions, rmse)

In [ ]:
# Your code here


You should get a lower RMSE in comparison to the use of ARIMA.

## 08 - Machine Learning Approach

We will now fit a non linear model such as a random forest. The idea is to predict a value from the last ones. You will try to create new columns in `df` that are shifted version of `df['Sales']`. Do it with a shift from 1 to 12.


In [ ]:
# Your code here


Create a random forest algorithm and use only the shifted version of the time series that you just created. You can use the following function to test it. It takes the true y values (`data`), the indexes of the test samples (`test_indexes`), the predictor (`predictor`: your random forest algorithm) and the shifted columns (`full_X`).

In [ ]:
# For each predictor, design a method to evaluate its performance on the test set:
def evaluate_performance_month_prediction_lm_rf(data, test_indexes, predictor, full_X):
    predictions = []
    ground_truth = []
    i = 0
    for _indx in test_indexes:
        train_data = data[:_indx]
        current_ground_truth = data[_indx]
        current_ground_truth_features = full_X[_indx,:]
        train_features = full_X[:_indx]
        predictor.fit(train_features, train_data)
        prediction = predictor.predict(current_ground_truth_features.reshape(1,-1))[0]
        predictions.append(prediction)
        ground_truth.append(current_ground_truth)
        i += 1
    rmse = np.sqrt(mean_squared_error(ground_truth, predictions))
    return ground_truth, predictions, rmse

In [ ]:
# Your code here


To improve our model, we can do some feature engineering. You will add moving averages to the data used to train the random forest.

Try to create 3 new columnns in `df`: one which is the rolling average of `df[Sales]` with a window of 12, one with a window of 3, and one with a window of 2. This will have the effect to isolate the trend and allow the algorithm to learn it. Also, plot these data.


In [ ]:
# Your code here


You can also add a more smoothing predictor using the exponential moving average (hint: method `.ewm` with `halflife` of 2, 3 and 12), that statistically optimizes an AR process. Plot also the data.

In [ ]:
# Your code here


In [ ]:
# Your code here


You should see a better RMSE!

Try to look at the importance of each feature. What do you find?


In [ ]:
# Your code here
